In [ ]:
import os
import glob
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
filename_disp_1 = 'disparity_map_exp_30000.npy'
filename_disp_2 = 'disparity_map_exp_40000.npy'
# load disparity map from numpy file
disparity_map_1 = np.load(f'../output/disp/{filename_disp_1}')
disparity_map_2 = np.load(f'../output/disp/{filename_disp_2}')

In [ ]:
# Calculate difference map
diff_map = disparity_map_2 - disparity_map_1
# discard nan and inf values for statistics
diff_map = np.nan_to_num(diff_map, nan=0.0, posinf=0.0, neginf=0.0)
print(f"Max difference: {np.max(diff_map)}, Min difference: {np.min(diff_map)}")
abs_diff_map = np.abs(diff_map)
print(f"Max absolute difference: {np.max(abs_diff_map)}, Min absolute difference: {np.min(abs_diff_map)}")

# discard outliers for better visualization
abs_diff_map = np.clip(abs_diff_map, 0, np.percentile(abs_diff_map, 99))
diff_map = np.clip(diff_map, np.percentile(diff_map, 1), np.percentile(diff_map, 99))

# Display original maps and difference
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# Original disparity maps
im1 = axes[0,0].imshow(disparity_map_1, cmap='jet')
axes[0,0].set_title(f'Depth Map {filename_disp_1}')
plt.colorbar(im1, ax=axes[0,0], label='Depth (m)')

im2 = axes[0,1].imshow(disparity_map_2, cmap='jet')
axes[0,1].set_title(f'Depth Map {filename_disp_2}')
plt.colorbar(im2, ax=axes[0,1], label='Depth (m)')

# Difference maps
im3 = axes[1,0].imshow(diff_map, cmap='RdBu', vmin=np.min(diff_map), vmax=np.max(diff_map))
axes[1,0].set_title('Difference Map')
plt.colorbar(im3, ax=axes[1,0], label='Depth diff (m)')

im4 = axes[1,1].imshow(abs_diff_map, cmap='hot')
axes[1,1].set_title('Absolute Difference Map')
plt.colorbar(im4, ax=axes[1,1], label='Depth diff (m)')

plt.tight_layout()
plt.show()

In [ ]:
# Statistical analysis of differences (inf-safe)
finite_mask = np.isfinite(diff_map) & np.isfinite(abs_diff_map)
diff_finite = diff_map[finite_mask]
abs_diff_finite = abs_diff_map[finite_mask]

print("=== Disparity Map Comparison Statistics ===")
print(f"Mean difference: {np.mean(diff_finite):.4f}")
print(f"Standard deviation of difference: {np.std(diff_finite):.4f}")
print(f"Mean absolute difference: {np.mean(abs_diff_finite):.4f}")
print(f"Maximum absolute difference: {np.max(abs_diff_finite):.4f}")
print(f"Minimum absolute difference: {np.min(abs_diff_finite):.4f}")

In [ ]:
def filter_outliers(data, lower_percentile=1, upper_percentile=99):
    """Remove outliers from data using percentile clipping."""
    finite_data = data[np.isfinite(data)]
    if len(finite_data) == 0:
        return finite_data
    
    lower_bound = np.percentile(finite_data, lower_percentile)
    upper_bound = np.percentile(finite_data, upper_percentile)
    return finite_data[(finite_data >= lower_bound) & (finite_data <= upper_bound)]

def compute_difference_stats(disparity_map, baseline_map):
    """Compute difference statistics between two disparity maps."""
    diff_map = disparity_map - baseline_map
    abs_diff_map = np.abs(diff_map)
    
    # Filter outliers
    filtered_abs = filter_outliers(abs_diff_map)
    filtered_signed = filter_outliers(diff_map)
    
    if len(filtered_abs) == 0:
        return None
    
    return {
        'abs_diff': filtered_abs,
        'signed_diff': filtered_signed,
        'outliers_removed': len(abs_diff_map[np.isfinite(abs_diff_map)]) - len(filtered_abs)
    }

def plot_percentile_differences(disparity_files, baseline_file, baseline_map, plot_type='absolute'):
    """Plot percentile differences from baseline."""
    percentiles = np.linspace(0, 100, 100)
    fig, ax = plt.subplots(figsize=(15, 8))
    
    for file_path in disparity_files:
        if file_path == baseline_file:
            continue
            
        disparity_map = np.load(file_path)
        stats = compute_difference_stats(disparity_map, baseline_map)
        
        if stats is None:
            continue
            
        # Choose data based on plot type
        data = stats['abs_diff'] if plot_type == 'absolute' else stats['signed_diff']
        perc_values = np.percentile(data, percentiles)
        
        filename = os.path.basename(file_path)
        ax.plot(percentiles, perc_values, 'o-', alpha=0.7, label=filename, linewidth=2, markersize=3)
    
    # Add zero line for signed differences
    if plot_type == 'signed':
        ax.axhline(y=0, color='black', linestyle='--', alpha=0.5, label='Zero difference')
    
    # Set labels and formatting
    ylabel = f"{'Absolute' if plot_type == 'absolute' else 'Signed'} Depth Difference from Baseline (outliers removed)"
    title = f"{'Absolute' if plot_type == 'absolute' else 'Signed'} Depth Differences from Baseline ({os.path.basename(baseline_file)})"
    
    ax.set_xlabel('Percentile')
    ax.set_ylabel(ylabel)
    ax.set_title(title)
    ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()

def print_summary_statistics(disparity_files, baseline_file, baseline_map):
    """Print summary statistics for all files relative to baseline."""
    print(f"\n=== Variation Summary (relative to {os.path.basename(baseline_file)}, outliers removed) ===")
    
    for file_path in disparity_files:
        if file_path == baseline_file:
            continue
            
        disparity_map = np.load(file_path)
        stats = compute_difference_stats(disparity_map, baseline_map)
        
        if stats is None:
            continue
            
        filename = os.path.basename(file_path)
        print(f"\n{filename}:")
        print(f"  Mean signed difference: {np.mean(stats['signed_diff']):.4f}")
        print(f"  Mean abs difference: {np.mean(stats['abs_diff']):.4f}")
        print(f"  Max abs difference: {np.max(stats['abs_diff']):.4f}")
        print(f"  95th percentile abs diff: {np.percentile(stats['abs_diff'], 95):.4f}")
        print(f"  Std deviation: {np.std(stats['signed_diff']):.4f}")
        
        total_finite = len(stats['abs_diff']) + stats['outliers_removed']
        outlier_percentage = 100 * stats['outliers_removed'] / total_finite if total_finite > 0 else 0
        print(f"  Outliers removed: {stats['outliers_removed']} / {total_finite} ({outlier_percentage:.1f}%)")

In [ ]:

# Main analysis code
disparity_files = glob.glob('../output/disp/disparity_map_*.npy')
disparity_files.sort()
filename_disp_baseline = 'exp_35000'

# Find the baseline file
baseline_file = next((f for f in disparity_files if filename_disp_baseline in f), None)

if baseline_file is None:
    print(f"Baseline file {filename_disp_baseline} not found!")
else:
    print(f"Using baseline: {os.path.basename(baseline_file)}")
    
    # Load baseline disparity map
    baseline_map = np.load(baseline_file)
    
    # Create plots
    plot_percentile_differences(disparity_files, baseline_file, baseline_map, 'absolute')
    plot_percentile_differences(disparity_files, baseline_file, baseline_map, 'signed')
    
    # Print statistics
    print_summary_statistics(disparity_files, baseline_file, baseline_map)